In [1]:
from rich import pretty, print, inspect
from tqdm.auto import tqdm
pretty.install()
from data_generation import data_generation
from model_builder import *

2024-02-25 07:59:48.126316: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df, data = data_generation()
display(df.head())
df  = df[-3000:]
df.head()

,Date,White Ball 0,White Ball 1,White Ball 2,White Ball 3,White Ball 4,White Ball 5,Grey Ball,Day,dayofweek,year,month,quarter,is_leap_year,weekofyear
165,1985-09-11,4,9,14,15,44,45,32,1187,2,1985,9,3,0,37
164,1985-09-14,11,23,43,46,47,49,1,1190,5,1985,9,3,0,37
163,1985-09-18,8,12,19,33,41,45,24,1194,2,1985,9,3,0,38
162,1985-09-21,3,11,16,20,21,24,44,1197,5,1985,9,3,0,38
161,1985-09-25,5,9,12,33,45,47,32,1201,2,1985,9,3,0,39


,Date,White Ball 0,White Ball 1,White Ball 2,White Ball 3,White Ball 4,White Ball 5,Grey Ball,Day,dayofweek,year,month,quarter,is_leap_year,weekofyear
1207,1995-05-17,1,3,7,27,34,41,21,4722,2,1995,5,2,0,20
1206,1995-05-20,5,12,24,29,38,41,47,4725,5,1995,5,2,0,20
1205,1995-05-24,2,5,9,14,44,46,42,4729,2,1995,5,2,0,21
1204,1995-05-27,18,19,29,31,35,43,10,4732,5,1995,5,2,0,21
1203,1995-05-31,17,25,27,30,32,36,49,4736,2,1995,5,2,0,22


In [6]:
windows, labels = transform_data(data)
print(f"Shape of windows is: {windows.shape}.")
print(f"Shape of labels is: {labels.shape}.")
display(windows[0], labels[0])

cols_to_keep = [
    'Day',
    'White Ball 0', 'White Ball 1', 'White Ball 2', 'White Ball 3',
       'White Ball 4', 'White Ball 5', 'Grey Ball',]

df = df.sort_values('Day')
df['Day'] = df['Day'] - df.Day.min()

data = df[cols_to_keep]
windows, labels = transform_data(data)
print(f"Shape of windows is: {windows.shape}.")
print(f"Shape of labels is: {labels.shape}.")
display(windows[0], labels[0])

Shape of windows is: (2994, 6, 8).

Shape of labels is: (2994, 6).

array([[4722,    1,    3,    7,   27,   34,   41,   21],
       [4725,    5,   12,   24,   29,   38,   41,   47],
       [4729,    2,    5,    9,   14,   44,   46,   42],
       [4732,   18,   19,   29,   31,   35,   43,   10],
       [4736,   17,   25,   27,   30,   32,   36,   49],
       [4739,   19,   33,   34,   35,   40,   41,   12]])

array([13, 14, 28, 33, 34, 48])

Shape of windows is: (2994, 6, 8).

Shape of labels is: (2994, 6).

array([[ 0,  1,  3,  7, 27, 34, 41, 21],
       [ 3,  5, 12, 24, 29, 38, 41, 47],
       [ 7,  2,  5,  9, 14, 44, 46, 42],
       [10, 18, 19, 29, 31, 35, 43, 10],
       [14, 17, 25, 27, 30, 32, 36, 49],
       [17, 19, 33, 34, 35, 40, 41, 12]])

array([13, 14, 28, 33, 34, 48])

In [8]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

results = dict(Horizon=[], Accuracy=[])


for horizon in tqdm(range(6, 20, 3)):
    X, y = transform_data(data, horizon=horizon)
    X = X.reshape(-1, X.shape[1] * 8)


    # Split the target variable into multiple target variables (assuming y has shape (4004, 6))
    # You may need to adjust this based on your specific dataset
    y1, y2, y3, y4, y5, y6 = y[:, 0], y[:, 1], y[:, 2], y[:, 3], y[:, 4], y[:, 5]

    # Split the dataset into training and testing sets
    X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test, y4_train, y4_test, y5_train, y5_test, y6_train, y6_test = train_test_split(
        X, y1, y2, y3, y4, y5, y6, test_size=0.2, random_state=42, shuffle=False)

    # label encode the dataset

    from sklearn.preprocessing import LabelEncoder


    # Define and train multiple XGBoost models
    models = []
    les = []
    for y_train, y_test in [
        (y1_train, y1_test), (y2_train, y2_test), (y3_train, y3_test), (y4_train, y4_test), (y5_train, y5_test), (y6_train, y6_test)
        ]:
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
    
        model = xgb.XGBClassifier(
            objective='multi:softmax', 
            booster='gbtree', 
            n_estimators=100, 
            max_depth=None, 
            learning_rate=0.1, 
            random_state=42)
        model.fit(X_train, y_train)
        models.append(model)
        les.append(le)


    # evaluation
    y_pred = np.array([le.inverse_transform(model.predict(X_test)) for model, le in zip(models, les)])
    eval = []
    for i, y_test in enumerate([y1_test, y2_test, y3_test, y4_test, y5_test, y6_test]):
        accuracy = accuracy_score(y_test, y_pred[i])
        # print(f"Accuracy: {accuracy}")
        eval.append(accuracy)
    print(f"Overall Accuracy for Horizon {horizon} is {sum(eval)/len(eval):.5f}")
    print("===================================================================\n\n\n")

    results['Horizon'].append(horizon)
    results['Accuracy'].append(sum(eval)/len(eval))


  0%|          | 0/5 [00:00<?, ?it/s]

Overall Accuracy for Horizon 6 is 0.06010

===================================================================

Overall Accuracy for Horizon 9 is 0.06066

===================================================================

...

===================================================================

Overall Accuracy for Horizon 15 is 0.05891

===================================================================

Overall Accuracy for Horizon 18 is 0.06896

===================================================================

NameError: name 'make_clf' is not defined